In [52]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [53]:
# import data
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# :: = some movies' title has a comma, so cant separate it with comma
# there is no header in the file, if not choose none, the first movie will be treated as a header.
# engine: make it more efficient
# encoding: some titles contain special char

In [13]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
users.head()
#1:user id
# 2: user age

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [16]:
ratings.head()
# 0: user id
# 1: movie id
# 2: rating

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [54]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
#we need array for pytorch
training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [55]:
training_set[:10]
# index: 0=user id, 1=movie id, 2=rating

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561],
       [        1,         8,         1, 875072484],
       [        1,         9,         5, 878543541],
       [        1,        11,         2, 875072262],
       [        1,        13,         5, 875071805],
       [        1,        15,         5, 875071608]])

In [56]:
test_set[:10]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883],
       [        1,        23,         4, 875072895],
       [        1,        24,         3, 875071713],
       [        1,        27,         2, 876892946],
       [        1,        31,         3, 875072144],
       [        1,        33,         4, 878542699]])

In [57]:
# find the totalnumber of users and movies
# user ids are in index 0 and movies aree 1
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [58]:
nb_users

943

In [59]:
nb_movies

1682

In [60]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users] # take all the movie ids([:,1]) of the first user([:,0] == id_users)
        id_ratings = data[:,2][data[:,0] == id_users]# take .... ratings ...
        ratings = np.zeros(nb_movies) # 1682 rates all 0 initially
        ratings[id_movies - 1] = id_ratings # id_movies start from 1. Then put the real ratings(=id_ratings)
        new_data.append(list(ratings))# make sure it is a list
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [61]:
#each list contain 1682 vlaues = 943 x 1682
# ratings of first user to the first 10 movies(the user likes the 9th movie)
training_set[0][:10]

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0]

In [62]:
test_set[0][:10]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0]

In [63]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [64]:
# converted into a much better looking array(943x1682)
training_set[:2]


    0     3     4  ...      0     0     0
    4     0     0  ...      0     0     0
[torch.FloatTensor of size 2x1682]

In [65]:
# Converting the ratings into binary ratings 1 = like and 0 = do not like
training_set[training_set == 0] = -1 # -1: did not watch the movie
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [66]:
class RBM():
    # nv: number of visible nodes. nh: number of hidden nodes
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv) # W: weights are initilize in torch tensor. randn gives it ND(mean 0 and var 1)
        
        # bias of the probabilit of hidden node given the visible nodes
        # the 1 will help to create a 2d tensor, nh will correspond to the bias
        self.a = torch.randn(1, nh) 
        self.b = torch.randn(1, nv) 
        
# compute the probability of the hidden node given the visible node. once obtain this probability, we can sample the
# activation activation of the hidden nodes 
# x will correspopnd to the visible neurons in the probability p(h|v)
    def sample_h(self, x):
        # mm: to make product of 2 tensors. The t means transpose(make sense because 2 same size matrix)
        wx = torch.mm(x, self.W.t()) 
        
        # expands will asure that each bias will apply to each mini batch
        activation = wx + self.a.expand_as(wx)
        
        # the probabilit of the hidden nodes will be activated according to the value of the visible nodes
        p_h_given_v = torch.sigmoid(activation)
        
        # each node in p_h_given_v (a vector) has a probabilit that will activate that hidden node
        # we want a list of vector for the hidden node ; 0 or 1. 0 if the probability is not high enough to activate the 
        # hidden node. 1 if high enough. thus we need bernoulli to do this.(beroulli sampling)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
# y correspond to the hidden node
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
# v0: input vector. vk: visible nodes obtained after k iterations
# ph0: vector of probabilities at the first iteration the hidden node=1 given the values of v0 
#phk: the probabilities of the hidden node after k sampling given the values of visible node -vk
    def train(self, v0, vk, ph0, phk):
        
        
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
                
        
nv = len(training_set[0]) # number of movies-1682
nh = 100 # optional number. 
batch_size = 100

# create an object for the class
rbm = RBM(nv, nh)
   

In [67]:
# Training the RBM
nb_epoch = 10
for epoch in range(1,(nb_epoch+1)):
    train_loss = 0
    s = 0. # a counter
    # update the weight after each batch of users 
    for id_user in range(0,(nb_users-batch_size),batch_size):
        vk=training_set[id_user:id_user+batch_size]# 0-100, then 100-200...
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0) # use sample_h function. ,_ to get the first value the function returns
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


        
        
        


/Users/michaelzeng/anaconda3/lib/python3.6/site-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)


epoch: 1 loss: 0.29332643051611795
epoch: 2 loss: 0.2502879714453166
epoch: 3 loss: 0.24989429585570122
epoch: 4 loss: 0.2513257225723896
epoch: 5 loss: 0.25014906895944
epoch: 6 loss: 0.25155015836604755
epoch: 7 loss: 0.24885791447165861
epoch: 8 loss: 0.24988272516675883
epoch: 9 loss: 0.25005101960849907
epoch: 10 loss: 0.25056638187503294


In [68]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
        
        
print('test loss: '+str(test_loss/s))

test loss: 0.24485796556443482


In [23]:
len(training_set[0])

1682